# PCM Modeling

In this section of the tutorial, we will use the knowledge from all previous tutorials to construct a simple PCM classification model for four  adenosine receptors (A1, A2A, A2B, A3).

## Data Preparation

First, we need to load the data. For this tutorial we will load the dataset containing data for all four adenosine receptors.

In [8]:
from qsprpred.data.data import MoleculeTable
import os

os.makedirs('../../tutorial_output/data', exist_ok=True)

dataset = MoleculeTable.fromTableFile(
    name='PCMTutorialDataset', 
    filename='../../tutorial_data/AR_LIGANDS.tsv',
    store_dir='../../tutorial_output/data',
)

dataset.getDF()

,SMILES,pchembl_value_Mean,accession,QSPRID
QSPRID,,,,
PCMTutorialDataset_0,Cc1nn(-c2cc(NC(=O)CCN(C)C)nc(-c3ccc(C)o3)n2)c(...,8.68,P29274,PCMTutorialDataset_0
PCMTutorialDataset_1,Cc1nn(-c2cc(NC(=O)CCN(C)C)nc(-c3ccc(C)o3)n2)c(...,6.68,P30542,PCMTutorialDataset_1
PCMTutorialDataset_2,Nc1c(C(=O)Nc2ccc([N+](=O)[O-])cc2)sc2c1cc1CCCC...,4.82,P29274,PCMTutorialDataset_2
PCMTutorialDataset_3,O=C(Nc1nc2ncccc2n2c(=O)n(-c3ccccc3)nc12)c1ccccc1,7.15,P0DMS8,PCMTutorialDataset_3
PCMTutorialDataset_4,O=C(Nc1nc2ncccc2n2c(=O)n(-c3ccccc3)nc12)c1ccccc1,5.65,P29274,PCMTutorialDataset_4
...,...,...,...,...
PCMTutorialDataset_12452,CCCOc1ccc(C=Cc2cc3c(c(=O)n(C)c(=O)n3C)n2C)cc1,6.51,P29274,PCMTutorialDataset_12452
PCMTutorialDataset_12453,CCOC(=O)c1cnc(NCC(C)C)n2nc(-c3ccco3)nc12,7.61,P0DMS8,PCMTutorialDataset_12453
PCMTutorialDataset_12454,CCOC(=O)c1cnc(NCC(C)C)n2nc(-c3ccco3)nc12,7.35,P29274,PCMTutorialDataset_12454


### Fetching Protein Data

In addition, it is also possible to easily fetch the sequences for our proteins from [Papyrus](https://jcheminf.biomedcentral.com/articles/10.1186/s13321-022-00672-x) data set. You can find more information on how to fetch data from Papyrus within QSPRpred in the [data collection](../../basics/data/data_collection_with_papyrus.ipynb) tutorial.

In [9]:
from qsprpred.data.sources.papyrus import Papyrus

# Papyrus database version 05.6
papyrus = Papyrus(
    data_dir="../../tutorial_data/papyrus",
    version='05.6',  # Papyrus database version
    stereo=False,
    plus_only=True,
)

# Get the protein sequences for the four adenosine receptors
acc_keys = ["P29274", "P29275", "P30542", "P0DMS8"] # A2AR, A2BR, A1R, A3R
dataset_seq = papyrus.getProteinData(acc_keys, name=f"{dataset.name}_seqs", use_existing=True)
dataset_seq

########## DISCLAIMER ##########
You are downloading the high-quality Papyrus++ dataset.
Should you want to access the entire, though of lower quality, Papyrus dataset,
look into additional switches of this command.
################################
Number of files to be downloaded: 3
Total size: 33.0MB


,target_id,HGNC_symbol,UniProtID,Status,Organism,Classification,Length,Sequence,TID,accession
0,P29275_WT,ADORA2B,AA2BR_HUMAN,reviewed,Homo sapiens (Human),Membrane receptor->Family A G protein-coupled ...,332,MLLETQDALYVALELVIAALSVAGNVLVCAAVGTANTLQTPTNYFL...,ChEMBL:CHEMBL255;ChEMBL:CHEMBL255;ChEMBL:CHEMB...,P29275
1,P30542_WT,ADORA1,AA1R_HUMAN,reviewed,Homo sapiens (Human),Membrane receptor->Family A G protein-coupled ...,326,MPPSISAFQAAYIGIEVLIALVSVPGNVLVIWAVKVNQALRDATFC...,ChEMBL:CHEMBL226;ChEMBL:CHEMBL226;ChEMBL:CHEMB...,P30542
2,P29274_WT,ADORA2A,AA2AR_HUMAN,reviewed,Homo sapiens (Human),Membrane receptor->Family A G protein-coupled ...,412,MPIMGSSVYITVELAIAVLAILGNVLVCWAVWLNSNLQNVTNYFVV...,ChEMBL:CHEMBL251;ChEMBL:CHEMBL251;ChEMBL:CHEMB...,P29274
3,P0DMS8_WT,ADORA3,AA3R_HUMAN,reviewed,Homo sapiens (Human),Membrane receptor->Family A G protein-coupled ...,318,MPNNSTALSLANVTYITMEIFIGLCAIVGNVLVICVVKLNPSLQTT...,ChEMBL:CHEMBL256;ChEMBL:CHEMBL256;ChEMBL:CHEMB...,P0DMS8


The  keys used to fetch the data are saved in the `accession` colum in the resulting data frame:

In [10]:
dataset.getSubset("accession")

,accession
QSPRID,
PCMTutorialDataset_0,P29274
PCMTutorialDataset_1,P30542
PCMTutorialDataset_2,P29274
PCMTutorialDataset_3,P0DMS8
PCMTutorialDataset_4,P29274
...,...
PCMTutorialDataset_12452,P29274
PCMTutorialDataset_12453,P0DMS8
PCMTutorialDataset_12454,P29274


## Calculating Protein Descriptors

In this section, we will show how to connect the information about sequences with our data set and calculate protein descriptors using multiple sequence alignment and the `PCMDataSet` class from the `qsprpred.extra` package. First, let us convert the original data set saved in the `dataset` variable to a `PCMDataSet`:

In [ ]:
from qsprpred.extra.data.data import PCMDataSet

def sequence_provider(acc_keys):
    """
    A function that provides a mapping from accession key to a protein sequence.

    Args:
        acc_keys (list): Accession keys of the protein to get a sequences for.

    Returns:
        (dict) : Mapping of accession keys to protein sequences.
        (dict) : Additional information to pass to the MSA provider (can be empty).
    """
    map = dict()
    info = dict()
    for i, row in dataset_seq.iterrows():
        map[row['accession']] = row['Sequence']

        # can be omitted
        info[row['accession']] = {
            'Organism': row['Organism'],
            'UniProtID': row['UniProtID'],
        }

    return map, info

dataset = PCMDataSet.fromMolTable(
    dataset,
    name=dataset.name,
    protein_col="accession",
    protein_seq_provider=sequence_provider,
    store_dir=dataset.outDir,
    target_props=dataset.targetProperties
)
dataset

`PCMDataset` knows how to connect accession keys to sequences thanks to `proteinCol` and `proteinSeqProvider`, and it will automatically create a multiple sequence alignment (MSA) for us when calculating protein descriptors, which is facilitated through the `addProteinDescriptors` method and a `ProteinDescriptorCalculator`:

In [ ]:
from qsprpred.extra.data.utils.descriptorcalculator import ProteinDescriptorCalculator
from qsprpred.extra.data.utils.descriptorsets import ProDec
from qsprpred.extra.data.utils.descriptor_utils.msa_calculator import ClustalMSA

calc = ProteinDescriptorCalculator(
    desc_sets=[ProDec(sets=["Zscale Hellberg"])],
    msa_provider=ClustalMSA(out_dir=dataset.storeDir)
)
dataset.addProteinDescriptors(calc)

We can check the descriptor matrix:

In [ ]:
dataset.getDescriptors()

We can of course combine this with molecular descriptors as well:

In [ ]:
from qsprpred.data.utils.descriptorsets import RDKitDescs
from qsprpred.data.utils.descriptorcalculator import MoleculeDescriptorsCalculator

dataset.nJobs = 12 # speeding things up a little
calc = MoleculeDescriptorsCalculator(desc_sets=[RDKitDescs()])
dataset.addDescriptors(calc)
dataset.getDescriptors()

## Model Construction

Now that we have defined our preprocessing steps, we can construct our model. We will use a simple random forest model in this case, but notice that the training code is allmost the same as in the simple [training tutorial](./tutorial_training.ipynb). The only difference is that we are using the `SklearnPCMModel` class instead of `SklearnModel`:

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from qsprpred.extra.models.pcm import SklearnPCMModel
from qsprpred.models.assessment_methods import CrossValAssessor, TestSetAssessor
from qsprpred.models.metrics import SklearnMetric

os.makedirs('../../tutorial_output/data', exist_ok=True)

model = SklearnPCMModel(
    base_dir = '../../tutorial_output/models',
    data=dataset,
    alg = KNeighborsRegressor(),
    name='PCMTutorialModel',
    random_state=42
)

score_func = SklearnMetric.getDefaultMetric(model.task)
CrossValAssessor(scoring = score_func)(model)
TestSetAssessor(scoring = score_func)(model)
model.fitAttached()

The `SklearnPCMModel` class is a subclass of `SklearnModel` and thus inherits all of its functionality. The only difference is in how it handles predictions, which we will [show later](#model-prediction). Model evaluation and plotting is also the same as in the [training tutorial](./tutorial_training.ipynb):

In [ ]:
from qsprpred.plotting.regression import CorrelationPlot

plot = CorrelationPlot([model])
plot.make(save=True, show=True, property_name="pchembl_value_Mean")

## Model Prediction

Prediction with the PCM model works similarly to regular models. We can load the saved model using its name (`RF_PCM`) as usual:

In [ ]:
model_from_files = SklearnPCMModel(name='PCMTutorialModel', base_dir = '../../tutorial_output/models')
model_from_files

But naturally when doing the prediction, we need to provide the protein ID to fetch protein descriptors for as well. You can see that regular call to `predict` now does not work

In [ ]:
sample_mols = [
    'CN1C=NC2=C1C(=O)N(C(=O)N2C)C', # caffeine
    'c1ccccc1CCN1N=CC2=C1N=C(N)N3C2=NC(C4=CC=CO4)=N3' # SCH-58261 (50x more selective to A2A than caffeine -> more potent)
]

In [ ]:
try:
    model_from_files.predictMols(
        sample_mols
    )
except TypeError as e:
    print(e)

We need to use one of the protein Idataset from the training data to fetch the appropriate protein descriptors:

In [ ]:
dataset.getSubset(dataset.proteinCol)[dataset.proteinCol].unique()

In [ ]:
model_from_files.predictMols(
    sample_mols,
    protein_id = 'P29274', # human adenosine A2A receptor
    use_probas=True
)

Now that makes sense. We can see that for the selective compound (`SCH-58261`), the model predicts it as active with confidence. If we use the same compoundataset with the other isoforms, we can see that indeed the result would be inactive for this compound hinting at its selectivity:

In [ ]:
model_from_files.predictMols(
    sample_mols,
    protein_id = 'P30542', # human adenosine A1 receptor
    use_probas=True
)

In [ ]:
model_from_files.predictMols(
    sample_mols,
    protein_id = 'P0DMS8', # human adenosine A3 receptor
    use_probas=True
)

In [ ]:
model_from_files.predictMols(
    sample_mols,
    protein_id = 'P29275', # human adenosine A2B receptor
    use_probas=True
)